<a href="https://colab.research.google.com/github/OvaisMemon/LangChain/blob/main/Project_3_Function_Calling_With_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project 3: LangChain Function/Tool Calling Calculator Project**

In [1]:
!pip install -Uq langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 12.9 MB/s eta 0:00:00


In [2]:
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
instruction = "You are a helpful Mathematics assistant. Perform calculations based on the user provided inputs. Only use the given tools to perform the operations and if user asks any other operation, then reply with 'Operation Not Supported'."
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY, system_instructions=instruction)

In [4]:
def add_numbers(num1:int, num2:int) -> int:
  """ Use this function to add two numbers given by user """
  """ num1: first integer """
  """ num2: second integer """
  return num1 + num2

def subtract_numbers(num1:int, num2:int) -> int:
  """ Use this function to subtract two numbers given by user. Always subtract the small number from the bigger one. """
  """ num1: integer """
  """ num2: integer """
  return abs(num1 - num2)

def multiply_numbers(num1:int, num2:int) -> int:
  """ Use this function to multiply two numbers given by user. """
  """ num1: integer """
  """ num2: integer """
  return num1 * num2

def divide_numbers(num1:int, num2:int) -> int:
  """ Use this function to divide two numbers given by user. """
  """ num1: integer """
  """ num2: integer """
  return num1 / num2

In [5]:
from langchain.tools import Tool
import re

add_tool = Tool(
    name="Addition",
    func=lambda x: "The result is: {}".format(add_numbers(*map(int, re.findall(r'\d+', x)))),
    description="Calculates the sum of two numbers. Input will be the two numbers within the user input."
)

sub_tool = Tool(
    name="Subtraction",
    func=lambda x: "The result is: {}".format(subtract_numbers(*map(int, re.findall(r'\d+', x)))),
    description="Calculates the difference of two numbers. Input will be the two numbers within the user input"
)

mul_tool = Tool(
    name="Multiplication",
    func=lambda x: "The result is: {}".format(multiply_numbers(*map(int, re.findall(r'\d+', x)))),
    description="Calculates the product of two numbers. Input will be the two numbers within the user input"
)

div_tool = Tool(
    name="Division",
    func=lambda x: "The result is: {}".format(divide_numbers(*map(int, re.findall(r'\d+', x)))),
    description="Calculates the division of two numbers. Input will be the two numbers within the user input"
)

tools = [add_tool, sub_tool, mul_tool, div_tool]

In [6]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,                                  # Provide the tools
    llm=llm,                                      # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                                  # Enable debugging output
)

<ipython-input-6-f0b364317b2f>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


**1. Ask direct input**

In [7]:
# Use the agent to solve a task that requires the tool
user_input = "Can you tell me the division of 56 with 7?"
agent.run(user_input)

<ipython-input-7-d6536816023e>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(user_input)




> Entering new AgentExecutor chain...
Question: Can you tell me the division of 56 with 7?
Thought: I need to divide 56 by 7 to find the answer.
Action: Division
Action Input: 56 7
Observation: The result is: 8.0
Thought:Question: Can you tell me the division of 56 with 7?
Thought: I need to divide 56 by 7 to find the answer.
Action: Division
Action Input: 56 7
Observation: The result is: 8.0
Thought:Question: Can you tell me the division of 56 with 7?
Thought: I need to divide 56 by 7 to find the answer.
Action: Division
Action Input: 56 7
Observation: The result is: 8.0
Thought:Question: Can you tell me the division of 56 with 7?
Thought: I need to divide 56 by 7 to find the answer.
Action: Division
Action Input: 56 7
Observation: The result is: 8.0
Thought:Question: Can you tell me the division of 56 with 7?
Thought: I need to divide 56 by 7 to find the answer.
Action: Division
Action Input: 56 7
Observation: The result is: 8.0
Thought:Question: Can you tell me the division of 56 

'Agent stopped due to iteration limit or time limit.'

**2. Ask with message template**

In [8]:
messages = [
    (
        "system",
        "You are a mathematician. Perform the calculation as requested by the user using given tools. Don't suggest the answer if the input not falls in any of the provided tools. Instead reply with 'Operation not supportd' ",
    ),
    ("human", "What is the result of 55 % 7"),
]

agent.run(messages)



> Entering new AgentExecutor chain...
Thought:The question asks for the modulo operation, which is not among the provided tools.
Final Answer: Operation not supported

> Finished chain.


'Operation not supported'

**3. Ask with Chat Prompt Template**

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = instruction

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{user_input}")]
)

query = "Can you tell me the product of 5 and 7?"
prompt = prompt_template.invoke({"user_input": query})

agent.run(prompt)



> Entering new AgentExecutor chain...
Thought: The question asks for the product of 5 and 7.  This requires multiplication.
Action: Multiplication
Action Input: 5, 7
Observation: The result is: 35
Thought:Question: messages=[SystemMessage(content="You are a helpful Mathematics assistant. Perform calculations based on the user provided inputs. Only use the given tools to perform the operations and if user asks any other operation, then reply with 'Operation Not Supported'.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you tell me the product of 5 and 7?', additional_kwargs={}, response_metadata={})]
Thought: The question asks for the product of 5 and 7.  This requires multiplication.
Action: Multiplication
Action Input: 5, 7
Observation: The result is: 35
Thought:Question: messages=[SystemMessage(content="You are a helpful Mathematics assistant. Perform calculations based on the user provided inputs. Only use the given tools to perform the operations and if 

Question: messages=[SystemMessage(content="You are a helpful Mathematics assistant. Perform calculations based on the user provided inputs. Only use the given tools to perform the operations and if user asks any other operation, then reply with 'Operation Not Supported'.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you tell me the product of 5 and 7?', additional_kwargs={}, response_metadata={})]
Thought: The question asks for the product of 5 and 7. This requires multiplication.
Action: Multiplication
Action Input: 5, 7
Observation: The result is: 35
Thought:Thought: I now know the final answer
Final Answer: 35

> Finished chain.


'35'